### Importing libraries and Downloading datasets

In [1]:
#Loading the libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Downloading and unzipping the required files
!wget -O ml-latest-small.zip https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip -o -j ml-latest-small.zip

--2021-08-11 18:20:42--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.59MB/s    in 0.2s    

2021-08-11 18:20:42 (4.59 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
  inflating: links.csv               
  inflating: tags.csv                
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: movies.csv              


### Reading datasets

In [3]:
#Reading 'ratings.csv' file
ratings_data = pd.read_csv('ratings.csv').drop('timestamp',1)

In [4]:
#Reading 'movies.csv' file
movies_data = pd.read_csv('movies.csv').drop('genres',1)

### Data Preprocessing

In [5]:
#Extracting 'year' from 'title' column and saving it in another column
movies_data['year'] = movies_data['title'].str.extract('(\(\d\d\d\d\))',expand=False)
movies_data['year'] = movies_data['year'].str.extract('(\d\d\d\d)',expand=False)

In [6]:
#Replacing all null values in 'year' column with 0
movies_data['year'] = movies_data['year'].replace(np.NaN,int(0))

#Typecasting the attributes into relevant datatypes
movies_data['year'] = [int(str(i).replace(",", "")) for i in movies_data['year']]

### Function implementation of Top-N Similar (User-Rated) Movies Recommender System (by Pearson Correlation)

---

Note: Dataset has movies released until 2018. So, you will get recommendations of movies released till 2018 only.

In [7]:
#Function definition for Top-N Similar Movie Recommendations (given year range and movie)
def get_top_similar_recommendations(n, from_year, to_year, movie):

  #Conditional statements for avoiding user to input anything out of bounds
  if from_year < 1850 and to_year > 2018:
    print("Please input year between 1850-2018 (both inclusive) !!!")
  elif int(movie[-5:-1]) < from_year or int(movie[-5:-1]) > to_year:
    print("Movie given as input must be under the requested year-range provided by user !!!")
  
  #If correct inputs mentioned, then implementing the function
  else:

    #Filtering the 'movies' dataset for given year-range
    movies_by_year = movies_data[(movies_data.year >= from_year) & (movies_data.year <= to_year)]

    #Merging 'movies_by_year' dataset with 'ratings' dataset
    movies_ratings = pd.merge(ratings_data, movies_by_year, on='movieId')

    #Ratings for movie and number of people who rated movie grouped by for each movie
    ratings_avg_count = pd.DataFrame(movies_ratings.groupby('title')['rating'].mean())
    ratings_avg_count.rename(columns={'rating':'ratings_avg'}, inplace=True)
    ratings_avg_count['ratings_count'] = pd.DataFrame(movies_ratings.groupby('title')['rating'].count())

    #Creating user-ratings matrix for each user giving rating to different movies
    movies_matrix = movies_ratings.pivot_table(index=['userId'], columns=['title'], values='rating')

    #Extracting user-ratings for movie provided by user
    movie_user_ratings =  movies_matrix[movie]

    #Extracting other similar movies using 'corrwith' function with 'Pearson Correlation' method
    #It checks pairwise similarity using Pearson Correlation Coefficient between user-ratings of input movie and other movies
    similar_movies = movies_matrix.corrwith(movie_user_ratings, method='pearson')

    #Creating dataframe for all the similar movies with their respective correlation score and all null values are dropped
    recommendations = pd.DataFrame(similar_movies, columns=['correlation_score'])
    recommendations.dropna(inplace = True)
    
    #Joining the count of people who rated movie with recommendations dataframe
    recommendations = recommendations.join(ratings_avg_count['ratings_count'])

    #Calculating Top 0.01% count of people who rated movies
    top_count = ratings_avg_count['ratings_count'].quantile(0.99)

    #Filtering the dataframe by removing all the recommendations below top 0.01% count
    #Sorting it in descending order by correlation score and selecting top-N + 1 recommendations (because first movie will be the same input movie)
    recommendations = recommendations[recommendations['ratings_count'] > top_count].sort_values(by='correlation_score', ascending = False).head(n+1)

    #Resetting the index of recommendations dataframe
    recommendations.reset_index(level=0, inplace=True)

    #Setting the columns' title for recommendations dataframe
    recommendations.columns = ['title','correlation_score','ratings_count']

    #Printing the statement for movie recommendations to users
    print(f'Users who watched {movie} also watched these movies released in {from_year}-{to_year}:' )

  #Returning top-N similar movie recommendations to function call
  #Excluding first movie which is the same as input movie since it has highest correlation score (of 1)
  return recommendations[1:]

In [8]:
#Function call for top-N Similar Movie Recommendations (by Pearson Correlation)
#(User need to input number of movies required as recommendations, from_year, to_year and movie)
#Also, user need to input movie in the format 'Movie_title (Release_year)'
get_top_similar_recommendations(10, 2005, 2015, 'Inception (2010)')

Users who watched Inception (2010) also watched these movies released in 2005-2015:


,title,correlation_score,ratings_count
1,"Dark Knight Rises, The (2012)",0.601706,76
2,"Prestige, The (2006)",0.587538,90
3,Batman Begins (2005),0.523953,116
4,"Hangover, The (2009)",0.507722,76
5,Interstellar (2014),0.491152,73
6,V for Vendetta (2006),0.454120,100
7,Slumdog Millionaire (2008),0.445741,71
8,Ratatouille (2007),0.421111,72
9,"Departed, The (2006)",0.414245,107
10,"Dark Knight, The (2008)",0.412281,149
